In [181]:
! ls *.py

credentials.py getFEC_v2.py   getPARTY_2.py
getFEC.py      getPARTY.py    run.py


In [182]:
import pandas as pd
import time

In [183]:
from credentials import *
import requests
import json, csv
import warnings
import random

from getFEC import *


def get_party_id(committee_id):
    stem = "https://api.open.fec.gov/v1/committee/"
    end = "{}/candidates/?sort=name&page=1&api_key={}&per_page=100".format(committee_id, api_key)
    url = stem+end

    data = get_url(url)['results']
    if len(data) <= 0:
        warnings.warn('WARNING: get_party_id search by committee id returns no results, try another method')
        return ("UNKNOWN PARTY ID")
    else:
        party_id = data[0]['party_full']
        return party_id


def get_committee_details(committee_id):
    stem = "https://api.open.fec.gov/v1/committee/"
    end = "{}/?sort=name&api_key={}&per_page=100&page=1".format(committee_id, api_key)
    url = stem+end

    data = get_url(url)['results']
    party_id = data[0]['party_full']
    candidate_ids = data[0]['candidate_ids']
    #office = data[0]['office_full']
    #name = data[0]['name']
    #return data[0]['party']
    return party_id, candidate_ids

def alt_receipt_id(rid, s, sample_frame):
    if rid == None:
        available = s
        next_sample = random.sample(set(sample_frame))
        r_new = r

def get_schedule_b_receipts(committee_id):
    #TODO way to specify year in URL?
    stem = "https://api.open.fec.gov/v1/committee/{}/schedules/".format(committee_id)
    end = "schedule_b/by_recipient_id/?per_page=20&api_key={}&page=1".format(api_key)
    url = stem+end

    data = get_url(url)['results']
    if len(data) <= 0:
        warnings.warn('WARNING: get_schedule_b_receipts search by committee id returns no results, try another method')
        return None
    elif len(data) > 0 and len(data) < 3:
        return data[0]['recipient_id']
    elif len(data) > 0 and len(data) >= 3:
        sample_frame = list(range(len(data)))
        s = random.sample(set(sample_frame), 3)
        print(s)
        print(data[s[0]], '\n', data[s[1]], '\n', data[s[2]], '\n')
        print(s[0], s[1], s[2])
        r1 = data[s[0]]['recipient_id']
        r2 = data[s[1]]['recipient_id']
        r3 = data[s[2]]['recipient_id']
        
        #TODO Replace If None
        print(r1, r2, r3)
        return [r1, r2, r3]
    else:
        pass

def find_schedule_b_results(committee_ids):
    if type(committee_ids) is list:
        cids = committee_ids
    else:
        cids = [committee_ids]
    assert len(cids) > 0

    print(cids)
    
    if len(cids) == 1:
        #print(cids)
        party_id = get_committee_details(cids[0])[0]
        return party_id
    else:

        pid_1 = get_committee_details(cids[0])[0]
        pid_2 = get_committee_details(cids[1])[0]
        pid_3 = get_committee_details(cids[2])[0]
        #print(pid_1, pid_2, pid_3)
        if (pid_1 == pid_2 == pid_3) is True:
            party_id = pid_1
            return party_id
        else:
            print("UNCLEAR Schedule B")
            print(pid_1, pid_2, pid_3)
            #return "UNCLEAR Schedule B Party ID"
            return(pid_1, pid_2, pid_3)


def search_party_id(committee_id):

    party_results = get_committee_details(committee_id)

    if party_results[0] is None and len(party_results[1]) == 0:
        print("no party results, further_tests_needed", party_results)
        print("[*] conducting schedule b search...")
        schedule_b_receipts = get_schedule_b_receipts(committee_id)
        party_id = find_schedule_b_results(schedule_b_receipts)
        return party_id

    elif party_results[0] is None and len(party_results[1]) > 0:
        print("candidate id exists but no party results", party_results)
        #try get_party_id search
        party_id = get_party_id(committee_id)
        return party_id

    elif party_results[0] is not None:
        print("party id found in committee details: ", party_results)
        party_id = party_results[0]
        return party_id

    else:
        #print("pass, unknown results", party_results)
        party_id = "UNKNOWN D"
        return party_id

#test_ids = ["C00401224", "C00000935", "C00464297", "C00571380"]
#test_manual_inf = ["dem pac", "dem comm", "rep can", "rep pac"]
#for cid  in test_ids:
#    party_id = search_party_id(cid)
#    print(party_id)


In [488]:
import numpy as np



def first_sample(n, sample_size):
    sample_frame = np.random.choice(n, size=n, replace=False, p=None)
    a = np.random.choice(n, size=sample_size, replace=False, p=None)
    return(sample_frame, a)


def resample_one(first_sample):

    sframe = first_sample[0]
    sample = first_sample[1]
    
    sframe = np.setdiff1d(sframe, sample)   
    sample = np.random.choice(sframe, size=1, replace=False, p=None)
    
    return(sframe, sample)


def all_samples(n, first_sample_size=3):

    s0 = first_sample(n, first_sample_size)
    sr = s0
    count = len(np.atleast_1d(sr[0]))
    samples = [s0[1].tolist()]

    while len(np.atleast_1d(sr[0])) != 1:
        if count == n:
            s1 = resample_one(s0)
            count -=1
            samples.append(s1[1].tolist())
            sr = (s1)
        else:
            sn = resample_one(sr)
            sr = (sn)
            samples.append(sn[1].tolist())     
    else:
        pass

    return(samples)
    
all_samples(10, 4)



[[8, 6, 5, 2], [3], [7], [0], [4], [1], [9]]

In [184]:
#extract party_id's from getFEC data


In [185]:
#df = pd.read_csv("2012__Boeing__schedule_a_merged.csv")

In [186]:
#committee_ids = df.committee_id.drop_duplicates()

In [187]:
#committee_ids.shape

In [188]:
#for id in committee_ids:
    

In [189]:
#for cid  in committee_ids:
#    party_id = search_party_id(cid)
#    time.sleep(1)
#    print(party_id)

In [190]:
test_ids = ["C00401224", "C00000935", "C00464297", "C00571380"]
test_manual_inf = ["dem pac", "dem comm", "rep can", "rep pac"]
for cid  in test_ids:
    party_id = search_party_id(cid)
    print(party_id)


no party results, further_tests_needed (None, [])
[*] conducting schedule b search...
[10, 9, 6]
{'committee_id': 'C00401224', 'count': 63, 'committee_name': 'ACTBLUE', 'total': 26878.31, 'recipient_id': 'C00625988', 'recipient_name': 'FUTURE FORUM PAC', 'cycle': 2018, 'idx': 758907} 
 {'committee_id': 'C00401224', 'count': 74830, 'committee_name': 'ACTBLUE', 'total': 1362767.16, 'recipient_id': 'C00562983', 'recipient_name': 'DEMOCRATIC ACTION', 'cycle': 2018, 'idx': 758726} 
 {'committee_id': 'C00401224', 'count': 1, 'committee_name': 'ACTBLUE', 'total': 1.0, 'recipient_id': None, 'cycle': 2018, 'idx': 1765830} 

10 9 6
C00625988 C00562983 None
['C00625988', 'C00562983', None]


IndexError: list index out of range